In [14]:
import ipyvuetify as v
import ipywidgets as widgets
import csv

In [15]:
#input list
file_input = None
file_name = None
country_selection = None
drawing_method = None
feature_collection = None

In [16]:
#output list 
asset_name = None
year = None

In [17]:
#map widget
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)

center = [0, 0]
zoom = 1

dc = DrawControl(marker={},
                 circlemarker={},
                 polyline={},
                 rectangle={'shapeOptions': {'color': '#0000FF'}},
                 circle={'shapeOptions': {'color': '#0000FF'}},
                 polygon={'shapeOptions': {'color': '#0000FF'}},
                 )

feature_collection = {
    'type': 'FeatureCollection',
    'features': []
}
def handle_draw(self, action, geo_json):
    global feature_collection
    feature_collection['features'].append(geo_json)

dc.on_draw(handle_draw)

m = Map(center=center, zoom=zoom)
#m.add_control(dc)
#m

In [18]:
#import file
widget_file_input = v.FileInput(label='select a file', v_model=None, class_="d-none")
#widget_file_input

widget_file_name = v.TextField(label='select a filename', v_model=None, class_="d-none")
#widget_file_name

In [ ]:
#country boundaries selector 
def create_FIPS_dic():
    """create the list of the country code in the FIPS norm using the CSV file provided in utils"""
    fips_dic = {}
    with open('../utils/FIPS_code_to_country.csv', newline='') as f:
        reader = csv.reader(f, delimiter=';')
        for row in reader:
            fips_dic[row[1]] = row[0]
        
    return fips_dic

widget_country_selection = v.Select(items=[*create_FIPS_dic()], label='Country/Province', v_model=None, class_="d-none")
#widget_country_selection

In [ ]:
#select the year of interest 
years = [2020, 2019]
widget_select_year = v.Select(items=years, label='Select a year', v_model=None)

def on_year_change(widget, event, data):
    global year
    year = widget.v_model

widget_select_year.on_event('change', on_year_change )
#widget_select_year

In [ ]:
#GEE asset
widget_asset_name = v.TextField(label='select a GEE asset', class_="d-none", v_model=None)

def on_asset_change(widget, event, data):
    global asset_name
    asset_name = widget.v_model
    
widget_asset_name.on_event('change', on_asset_change)
#widget_asset_name

In [ ]:
# add a method selector 
selection_method = ['Country boundaries', 'draw a shape', 'Upload file', 'use GEE asset']
#disableling the upload file at the moment 
widget_drawing_method = v.Select(items=selection_method, label='Select an AOI type', v_model='use GEE asset')

def on_method_change(widget, event, data):
    
    global selection_method
    global widget_file_input
    global widget_file_name
    global widget_country_selection
    global widget_asset_name
    global drawing_method
    
    if widget.v_model == selection_method[0]: 
        widget_file_input.class_= 'd-none'
        widget_file_name.class_='d-none'
        widget_country_selection.class_='d-inline'
        widget_asset_name.class_='d-none'
        m.remove_control(dc)
        dc.clear()
    elif widget.v_model == selection_method[1]:
        widget_file_input.class_= 'd-none'
        widget_file_name.class_='d-inline'
        widget_country_selection.class_='d-none'
        widget_asset_name.class_='d-none'
        m.add_control(dc)
    elif widget.v_model == selection_method[2]:
        widget_file_input.class_= 'd-inline'
        widget_file_name.class_='d-none'
        widget_country_selection.class_='d-none'
        widget_asset_name.class_='d-none'
        m.remove_control(dc)
        dc.clear()
    elif widget.v_model == selection_method[3]:
        widget_file_input.class_= 'd-none'
        widget_file_name.class_='d-none'
        widget_country_selection.class_='d-none'
        widget_asset_name.class_='d-inline'
        m.remove_control(dc)
        dc.clear()
        
    drawing_method = widget.v_model
    

widget_drawing_method.on_event('change', on_method_change)
#widget_drawing_method

In [ ]:
widget_alert = v.Alert(children=['turlututu'], type='info', text=True, class_="mt-5")
#widget_alert

In [ ]:
#validate the selected data 
widget_validate_data = v.Container(children=[
    v.Btn(color='primary', children=[
        v.Icon(left=True, children=[
            'mdi-map-marker-check'
        ]),
        'Select these inputs'
    ])
])

def on_click(widget, event, data):
    
    global widget_alert
    global drawing_method
    
    text = ['toto']
    widget_alert.children = text
    
    #do something 
    #asset = run_GLAD_input(file_input, file_name, country_selection, asset_name, drawing_method, feature_collection)

    #update the map with asset
    #updateMap(m, asset)

widget_validate_data.on_event('click', on_click)
#widget_validate_data

In [ ]:
#create the display 
inputs = v.Layout(
    _metadata={'mount-id': 'data-input'},
    row=True,
    class_="pa-5",
    align_center=True, 
    children=[
        v.Flex(xs12=True, children=[widget_drawing_method]),
        v.Flex(xs12=True, children=[widget_country_selection]),
        v.Flex(xs12=True, children=[widget_file_input]),
        v.Flex(xs12=True, children=[widget_file_name]),
        v.Flex(xs12=True, children=[widget_select_year]),
        v.Flex(xs12=True, children=[widget_validate_data]),
        v.Flex(xs12=True, children=[widget_alert]),
    ]
)
#inputs

AOI_content_main =  v.Layout(
    _metadata={'mount_id': 'content-main'},
    row=True,
    xs12=True,
    align_center=True, 
    class_="ma-5",
    children=[
        v.Card( 
            class_="pa-5",
            raised=True,
            xs12=True,
            children=[
                v.Html(xs12=True, tag='h2', children=['Select an AOI']),
                v.Row(
                    xs12=True,
                    children=[
                        v.Flex(xs12=True, lg6=True, children=[inputs]),
                        v.Flex(class_="pa-5", xs12=True, lg6=True, children=[m])
                    ]
                )    
            ]
        )
    ]
)
AOI_content_main

In [ ]:
import ee

#initialize earth engine
ee.Initialize()

def run_GLAD_input(file_input, file_name, country_selection, asset_name, drawing_method, feature_collection):
    #check the drawing method
    if drawing_method == None:
        alert_type = 'warning'
        message = 'No selection method have bee picked up'
        asset = None
    elif drawing_method == 'Country boundaries':
        if country_selection == None:
            alert_type = 'warning'
            message = 'No Country have been selected'
            asset = None
            asset_name = country_selection + '_boundaries'
        else:
            exist = False #verifier que l'asset n'existe pas 
            #si il existe 
            if exist:
                asset = asset_name
                alert_type = 'info'
                message = "The asset was already existing you can continue to use it"
            else:
                country_code = create_FIPS_dic()[country_selection]
                
            
    return (alert_type, message, asset)


#for testing purpose
#input list
file_input = None
file_name = None
country_selection = None
drawing_method = None
feature_collection = None
asset_name = None
year = None

#folder = ee.data.getAssetRoots()[0]['id']
#folder
#ee.data.listAssets({'parent': folder})


    